In [1]:
import PyPDF2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [2]:
df=pd.read_csv("Heart_minor.csv",index_col=0)
pdf = open("report4.pdf","rb")
reader = PyPDF2.PdfReader(pdf)

In [3]:
attributes = ["age","sex","chest pain","cholesterol","(fasting)","thalach","angina:","old-peak","slope","(ca)","thalassemia","target"]
df.columns = attributes
attributes = attributes[:-1]
print(attributes)

['age', 'sex', 'chest pain', 'cholesterol', '(fasting)', 'thalach', 'angina:', 'old-peak', 'slope', '(ca)', 'thalassemia']


In [4]:
text=""
page=reader.pages[0]
text=page.extract_text().lower()
text

' \ncontact:  0181 -7834932  www .drintelligent .com  phone : 8247238429  dr. intelligent adanced heart -care centre  \n \npatient details  \nname: patient intelligent                        age: 21;                        sex: m; \n \n \nheart -report  \n \nchest pain:  atypical angina  \n \ncholesterol:   233 mg/dl  \n \nsugar (fasting):  130 mg/dl  \n \nthalach:  150 bpm  \n \nangina:  yes \n \nold-peak:  2.3 \n \nslope:  down -sloping  \n \nmajor -vessels (ca): 0 \n \nthalassemia : fixed -defect  \n \n '

In [5]:
def find_value(i,text):
    index=text.find(i)
    index=index+len(i)
    end=index
    while text[end] != '\n':
        end+=1
    li_value = text[index:end]
    value_list = li_value.split() 
    if(value_list[0]==':'):
        value_list.remove(value_list[0])
    temp_value = value_list[0]
    temp_dict={"m":1,"f":0,"asymptomatic":0,"atypical":1,
               "non-anginal":2,"typical":3,"yes":1,"no":0,
               "down":0,"flat":1,"upsloping":2,"fixed":1,"normal":2,"reversible":3}
    if(temp_value[-1]==';'):
        temp_value=temp_value[:-1]
    new_temp_value=temp_value
    if(temp_value in list(temp_dict.keys())):
        new_temp_value=temp_dict[temp_value]
    return new_temp_value

In [6]:
values = {}
for i in attributes:
    if(i in text):
        value = find_value(i,text)
        values[i]=float(value)
    else:
        values[i]=-1
values

{'age': 21.0,
 'sex': 1.0,
 'chest pain': 1.0,
 'cholesterol': 233.0,
 '(fasting)': 130.0,
 'thalach': 150.0,
 'angina:': 1.0,
 'old-peak': 2.3,
 'slope': 0.0,
 '(ca)': 0.0,
 'thalassemia': 1.0}

In [7]:
for i in values.keys():
    if(values[i]==-1):
        df.drop(columns=[i],inplace=True)
df

,age,sex,chest pain,cholesterol,(fasting),thalach,angina:,old-peak,slope,(ca),thalassemia,target
0,63,1,3,233,1,150,0,2.3,0,0,1,1
1,37,1,2,250,0,187,0,3.5,0,0,2,1
2,41,0,1,204,0,172,0,1.4,2,0,2,1
3,56,1,1,236,0,178,0,0.8,2,0,2,1
4,57,0,0,354,0,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,241,0,123,1,0.2,1,0,3,0
299,45,1,3,264,0,132,0,1.2,1,0,3,0
300,68,1,0,193,1,141,0,3.4,1,2,3,0
301,57,1,0,131,0,115,1,1.2,1,1,3,0


In [8]:
x = df.iloc[:,:-1].values
y = df.iloc[:,-1].values
rand_int = np.random.randint(1,100)
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=rand_int)
ss = StandardScaler()
x_train_ss = ss.fit_transform(x_train)
x_test_ss = ss.transform(x_test)
KNN = KNeighborsClassifier(n_neighbors=13,metric="minkowski",p=2)
KNN.fit(x_train_ss,y_train)
y_pred = KNN.predict(x_test_ss)


In [9]:
input_list = list(map(float,[values[x] for x in values.keys() if(values[x])!=-1]))
input_list

[21.0, 1.0, 1.0, 233.0, 130.0, 150.0, 1.0, 2.3, 0.0, 0.0, 1.0]

In [10]:
label = KNN.predict(ss.transform([input_list]))
if(label==0):
    output="Don't Have"
if(label==1):
    output = "Have"
print(f"\nWith {round(accuracy_score(y_test,y_pred)*100,2)} accuracy our Doctor Believes that you {output} a Heart Disease.")


With 81.97 accuracy our Doctor Believes that you Have a Heart Disease.
